# Model

* **Team :** Data Crew

This is the corresponding jupyter notebook where the model will be developed and stored in order to make predictions on the data.

In [23]:
# Basic Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Own Libraries
from utils import has_correct_sequence, fingerhut_data_cleaner

sns.set_style('whitegrid')

## Retrieving the data

In [24]:
data = pd.read_csv('../../1. Data/smaller_sample.csv')
event_defs = pd.read_csv('../../1. Data/Event+Definitions.csv')

data

,customer_id,account_id,stage,ed_id,event_name,event_timestamp,journey_steps_until_end,journey_id,milestone_number
0,2.787130e+08,-1812321640,Discover,2,campaign_click,2021-05-31 06:00:00,1,1.0,NaN
1,2.787130e+08,-1812321640,Apply for Credit,19,application_web_view,2021-05-31 23:11:03,2,1.0,NaN
2,2.787130e+08,-1812321640,Apply for Credit,3,application_web_submit,2021-05-31 23:11:51,3,1.0,NaN
3,2.787130e+08,-1812321640,Apply for Credit,19,application_web_view,2021-05-31 23:11:51,4,1.0,NaN
4,2.787130e+08,-1812321640,Apply for Credit,19,application_web_view,2021-05-31 23:11:51,5,1.0,NaN
...,...,...,...,...,...,...,...,...,...
3246336,-1.948257e+09,-1123823706,First Purchase,5,view_cart,2023-08-29 02:45:54,28,1.0,NaN
3246337,-1.948257e+09,-1123823706,First Purchase,4,browse_products,2023-08-29 02:45:54,29,1.0,NaN
3246338,-1.948257e+09,-1123823706,First Purchase,11,add_to_cart,2023-08-29 02:45:54,30,1.0,NaN
3246339,-1.948257e+09,-1123823706,First Purchase,6,begin_checkout,2023-08-29 02:45:54,31,1.0,NaN


* Applying the cleaning functions

In [25]:
df = fingerhut_data_cleaner(data, event_defs)
df

,customer_id,account_id,ed_id,event_name,event_timestamp,ed_id,journey_steps_until_end,stage
0,2.787130e+08,-1812321640,2,campaign_click,2021-05-31 06:00:00,2,1,Discover
1,2.787130e+08,-1812321640,19,application_web_view,2021-05-31 23:11:03,19,2,Apply for Credit
2,2.787130e+08,-1812321640,3,application_web_submit,2021-05-31 23:11:51,3,3,Apply for Credit
3,2.787130e+08,-1812321640,19,application_web_view,2021-05-31 23:11:51,19,4,Apply for Credit
4,2.787130e+08,-1812321640,19,application_web_view,2021-05-31 23:11:51,19,5,Apply for Credit
...,...,...,...,...,...,...,...,...
3246336,-1.948257e+09,-1123823706,5,view_cart,2023-08-29 02:45:54,5,28,First Purchase
3246337,-1.948257e+09,-1123823706,4,browse_products,2023-08-29 02:45:54,4,29,First Purchase
3246338,-1.948257e+09,-1123823706,11,add_to_cart,2023-08-29 02:45:54,11,30,First Purchase
3246339,-1.948257e+09,-1123823706,6,begin_checkout,2023-08-29 02:45:54,6,31,First Purchase


* Verifying if the data is clean and the sequences are corrected.

In [26]:
has_correct_sequence(df['journey_steps_until_end'])

True

## Obtaining the sequences with the correct states for the model

In [27]:
df['ed_id']

,ed_id,ed_id
0,2,2
1,19,19
2,3,3
3,19,19
4,19,19
...,...,...
3246336,5,5
3246337,4,4
3246338,11,11
3246339,6,6
